In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

# ✅ Cargar el modelo guardado
modelo = tf.keras.models.load_model(
    'modelo_final/modelo_clip_count.h5',
    compile=False  # Evita recompilar si no vas a reentrenar
)

# ✅ Compilar el modelo (opcional si necesitas métricas)
modelo.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='mean_squared_error',
    metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')]
)

print("✅ Modelo cargado correctamente.")




✅ Modelo cargado correctamente.


In [2]:
# ✅ Rutas de Datos para Test
ruta_test_csv = r'D:\Archivos de usuarios\Mikel Telo\OneDrive\Documentos\TB-DS-BIO-23.09.24\REPOSITORIOS\Mikel\clip_count\test.csv'
ruta_test_img = r'D:\Archivos de usuarios\Mikel Telo\OneDrive\Documentos\TB-DS-BIO-23.09.24\REPOSITORIOS\Mikel\clip_count\test'

# ✅ Cargar datos de prueba
df_test = pd.read_csv(ruta_test_csv)

# ✅ Preprocesar imágenes
def preprocesar_imagen(id):
    img_path = tf.strings.join([ruta_test_img, "/clips-", tf.strings.as_string(id), ".png"])
    img = tf.io.read_file(img_path)
    img = tf.image.decode_png(img, channels=1)
    img = tf.image.resize(img, (128, 128))
    img = img / 255.0
    return img

dataset_test = tf.data.Dataset.from_tensor_slices(df_test['id'].values)
dataset_test = dataset_test.map(lambda id: preprocesar_imagen(id))
dataset_test = dataset_test.batch(32).prefetch(tf.data.AUTOTUNE)

# ✅ Realizar predicciones
predicciones = modelo.predict(dataset_test).flatten()

# ✅ Agregar predicciones al DataFrame
df_test['clip_count'] = np.round(predicciones).astype(int)

# ✅ Asegurar valores en rango esperado
df_test['clip_count'] = np.clip(df_test['clip_count'], 1, 74)

# ✅ Guardar el archivo de salida
df_test[['id', 'clip_count']].to_csv('submission_final.csv', index=False)
print("✅ Archivo 'submission_final.csv' generado correctamente.")

157/157 [==============================] - 17s 107ms/step
✅ Archivo 'submission_final.csv' generado correctamente.


In [3]:
# ✅ Evaluar el modelo
loss, rmse = modelo.evaluate(dataset_test)
print(f"🔹 Pérdida: {loss:.2f}")
print(f"🔹 RMSE: {rmse:.2f}")

157/157 [==============================] - 3s 17ms/step - loss: 0.0000e+00 - rmse: 0.0000e+00
🔹 Pérdida: 0.00
🔹 RMSE: 0.00
